The ultimate goal is to perform holoVQE. \
It is divided into two parts: first part include functions to calculate expectation value of H; the second part include functions for the optimization\
More specifically, part 1 includes: functions to construct MPS from the unitary matrices, functions to construct H as MPO and functions to calculate the expectation value of H.\
Part 1:

In [16]:
import tenpy.linalg.np_conserved as npc
from tenpy.networks.mpo import MPO, MPOEnvironment
from tenpy.networks.mps import MPS
from tenpy.networks.site import BosonSite, SpinHalfSite
from tenpy.models.lattice import Chain
from scipy.stats import unitary_group
from tenpy.networks.site import BosonSite
from tenpy.models.lattice import Chain
from tenpy.networks.mpo import MPO, MPOEnvironment

In [17]:
def Uni_to_ten(uni_matrix, site_num, d_bond, d_phys_index, L):
    '''
    construct the matrix corresponding to physics leg index in local Hilbert space for translational invariant 1d chain
    
    Parameters
    ----------
    uni_matrix: numpy.ndarray
        the unitary matrix already constructed
    site_num: int
        the position of the site
    d_bond: int
        bond dimension
    d_phys_index: int
        the index in the physical dimension
    L: int
        the total number of sites
    '''
    if site_num == 0:
        res_matrix = uni_matrix[0:1, d_phys_index * d_bond: (d_phys_index+1)*d_bond]
    elif site_num == L-1:
        res_matrix = uni_matrix[0:d_bond, d_phys_index * d_bond: d_phys_index * d_bond + 1]
    else:
        res_matrix = uni_matrix[0:d_bond, d_phys_index * d_bond: (d_phys_index + 1) * d_bond]
    return res_matrix

In [18]:
def Site_Ten(uni_matrix, site_num, d_bond, d_phys, L):
    '''
    construct the 3-leg tensor for one site of the translational invariant 1d chain
    
    Parameters
    ----------
    d_phys: dimension of local Hilbert space
    
    returns the npc.Array type, which contains the tensor and its leg labels
    '''
    res_tensor = npc.Array.from_ndarray_trivial([Uni_to_ten(uni_matrix, site_num, d_bond, i, L) \
                                                 for i in range(0,d_phys)],labels=['p'+str(site_num),'vL','vR'])
    return res_tensor

In [19]:
def MPS_state(UM_list, d_phys, d_bond, L):
    '''
    construct the Matrix Product state for the whole chain (1d translational invariant bosonic chain) (can be adjusted)
    
    Parameters
    ----------
    UM_list: list of numpy.ndarray matrices
    
    returns the MPS type in TenPy
    '''
    site = BosonSite(Nmax = 1, conserve = None)
    # site = SpinHalfSite(conserve = None)
    lat = Chain(L, site, bc_MPS = 'finite')
    state = Site_Ten(UM_list[0], 0, d_phys, d_bond, L)
    for i in range(1, L):
        state1 = Site_Ten(UM_list[i], i, d_phys, d_bond, L)
        state = npc.tensordot(state, state1, ['vL', 'vR'])
    psi = MPS.from_full(lat.mps_sites(), state, 'B')
    # other methods to consider, MPS.from_Bflat, MPS
    # problem with MPS.from_Bflat now
    return psi

In [20]:
def MPO_H_contract(H_mpo, psi, L):
    '''
    calculate the expectation value of H by doing the contraction over the full MPS

    Parameters
    ----------
    H_mpo: MPO type
    psi: MPS type
    L: integer
        the total number of sites
    '''
    env = MPOEnvironment(psi, H_mpo, psi)
    E = env.full_contraction(L-1)
    return E

test

In [21]:
x1 = unitary_group.rvs(4)
x2 = unitary_group.rvs(4)
x3 = unitary_group.rvs(4)
psi = MPS_state([x1,x2,x3],1,1,3)

In [24]:
L = 3
N_max = 1
# Nmax is the maximum boson at one site, for TFIM is 1; 
site = BosonSite(Nmax = N_max, conserve = None)
bX = (site.B + site.Bd) / 2
bY = (site.B - site.Bd) / (2 * 1j)
bZ = site.N - site.Id * N_max / 2
Id = site.Id

# construct the matrix/array which represents H_mpo: Ws
# transverse field ising model
g = 1.
J = 1.
d = 2
W_bulk = [[Id, bX, -g*bZ],
          [None, None, -J*bX],
          [None, None, Id]]
W_first = [W_bulk[0]]
W_last = [[row[-1]] for row in W_bulk]
Ws = [W_first] + [W_bulk] * (L - 2) + [W_last]
H = MPO.from_grids([site]*L, Ws, bc = 'finite', IdL = 0, IdR = -1) # keep IdL, IdR for now

print("<psi|H|psi> =", H.expectation_value(psi))

ValueError: incompatible LegCharge
self | other
 -1  |  -1  
0 [] | 0 [] 
2    | 1    